### STEP 1 RAW 데이터 불러오기
원본 train 데이터인 train.csv와 우리 팀이 생성한 일반데이터인 'general.csv'를 불러와 'train_general_combined.csv'로 병합합니다.


In [ ]:
import pandas as pd

# CSV 파일 불러오기
train = pd.read_csv("train.csv")     # 첫 번째 CSV
general = pd.read_csv("general.csv") # 두 번째 CSV

# 두 DataFrame 합치기 (train 다음에 general)
combined = pd.concat([train, general], ignore_index=True)

output_path = "train_general_combined.csv"
combined.to_csv(output_path, index=False)

print(f"✅ 병합 완료! 저장 경로: {output_path}")
print(f"총 행 개수: {len(combined)}")

✅ 병합 완료! 저장 경로: train_general_combined.csv
총 행 개수: 4750


In [ ]:

import numpy as np
import re
from typing import List
import random

# 랜덤 시드 고정 (재현성)
random.seed(42)
np.random.seed(42)

# ===================================================================
# 1. 데이터 로드
# ===================================================================

print("=" * 70)
print("1. 데이터 로드 및 기본 분석")
print("=" * 70)

df = pd.read_csv('./output/train_general_combined.csv')

print(f"\n전체 데이터 수: {len(df):,}개")
print(f"컬럼: {df.columns.tolist()}")
print(f"\n클래스 분포:")
print(df['class'].value_counts())

# 줄바꿈 패턴 분석
df['newline_count'] = df['conversation'].str.count('\n')
print(f"\n줄바꿈 통계:")
print(df['newline_count'].describe())
print(f"\n- 줄바꿈 없는 데이터: {(df['newline_count'] == 0).sum():,}개")
print(f"- 줄바꿈 있는 데이터: {(df['newline_count'] > 0).sum():,}개")

# ===================================================================
# 2. 전처리 함수 정의
# ===================================================================

print("\n" + "=" * 70)
print("2. 전처리 함수 정의")
print("=" * 70)

def preprocess_strategy_1(text: str) -> str:
    """
    전략 1: 줄바꿈 완전 제거 - 전체를 단일 문맥으로 통일
    
    - 모든 줄바꿈(\n)을 공백으로 대체
    - 여러 공백을 하나로 정리
    - 문장 간 자연스러운 흐름 유지
    """
    # 줄바꿈을 공백으로 대체
    processed = text.replace('\n', ' ')
    
    # 다중 공백을 단일 공백으로 정리
    processed = re.sub(r'\s+', ' ', processed)
    
    # 앞뒤 공백 제거
    processed = processed.strip()
    
    return processed


def preprocess_strategy_2(text: str, separator: str = '[SEP]') -> str:
    """
    전략 2: 줄바꿈을 구분 토큰으로 대체
    
    - 줄바꿈(\n)을 특수 구분 토큰(기본값: [SEP])으로 대체
    - 대화 턴의 구조 정보 보존
    - 모델이 턴 구분을 학습할 수 있도록 함
    """
    # 줄바꿈을 구분 토큰으로 대체
    processed = text.replace('\n', f' {separator} ')
    
    # 다중 공백 정리
    processed = re.sub(r'\s+', ' ', processed)
    
    # 앞뒤 공백 제거
    processed = processed.strip()
    
    return processed


def add_random_newlines(text: str, min_lines: int = 3, max_lines: int = 10) -> str:
    """
    전략 3: 일반 대화 데이터에 임의 줄바꿈 추가
    
    - 단일 문장 데이터를 여러 줄로 분할
    - 문장 부호(., ?, !) 기준으로 분할
    - 데이터 균형 확보 (train과 test 간 포맷 차이 완화)
    """
    # 이미 줄바꿈이 충분히 있으면 그대로 반환
    if text.count('\n') >= min_lines:
        return text
    
    # 문장 부호 기준으로 분할 가능한 위치 찾기
    sentences = re.split(r'([.?!]\s+)', text)
    
    # 분할 결과가 너무 적으면 원본 반환
    if len(sentences) < 3:
        return text
    
    # 문장과 구분자 재조합
    reconstructed = []
    for i in range(0, len(sentences)-1, 2):
        if i+1 < len(sentences):
            reconstructed.append(sentences[i] + sentences[i+1].strip())
        else:
            reconstructed.append(sentences[i])
    
    # 마지막 요소 처리
    if len(sentences) % 2 == 1:
        reconstructed.append(sentences[-1])
    
    # 줄바꿈 추가할 개수 결정
    num_newlines = min(random.randint(min_lines, max_lines), len(reconstructed)-1)
    
    # 랜덤하게 줄바꿈 삽입
    if len(reconstructed) > 1:
        # 줄바꿈 위치 랜덤 선택
        split_indices = sorted(random.sample(range(1, len(reconstructed)), 
                                           min(num_newlines, len(reconstructed)-1)))
        
        result_parts = []
        prev_idx = 0
        for idx in split_indices:
            result_parts.append(' '.join(reconstructed[prev_idx:idx]))
            prev_idx = idx
        result_parts.append(' '.join(reconstructed[prev_idx:]))
        
        return '\n'.join(result_parts)
    
    return text


# ===================================================================
# 3. 전처리 적용
# ===================================================================

print("\n" + "=" * 70)
print("3. 각 전략별 전처리 수행")
print("=" * 70)

# 전략 1: 줄바꿈 완전 제거
print("\n[전략 1] 줄바꿈 완전 제거 적용 중...")
df_strategy1 = df.copy()
df_strategy1['conversation_processed'] = df_strategy1['conversation'].apply(preprocess_strategy_1)
df_strategy1['strategy'] = 'strategy_1_remove_newlines'

# 전략 2: 구분 토큰으로 대체
print("[전략 2] 구분 토큰([SEP]) 대체 적용 중...")
df_strategy2 = df.copy()
df_strategy2['conversation_processed'] = df_strategy2['conversation'].apply(
    lambda x: preprocess_strategy_2(x, separator='[SEP]')
)
df_strategy2['strategy'] = 'strategy_2_separator_token'

# 전략 3: 일반 대화에 줄바꿈 추가
print("[전략 3] 일반 대화 데이터에 줄바꿈 추가 중...")
df_strategy3 = df.copy()

# 일반 대화(줄바꿈 없는 데이터)에만 줄바꿈 추가
mask_general = df_strategy3['newline_count'] == 0
df_strategy3.loc[mask_general, 'conversation_processed'] = df_strategy3.loc[mask_general, 'conversation'].apply(
    add_random_newlines
)
df_strategy3.loc[~mask_general, 'conversation_processed'] = df_strategy3.loc[~mask_general, 'conversation']
df_strategy3['strategy'] = 'strategy_3_add_random_newlines'

print("\n전처리 완료!")

# ===================================================================
# 4. 결과 분석 및 비교
# ===================================================================

print("\n" + "=" * 70)
print("4. 전처리 결과 분석")
print("=" * 70)

def analyze_processed_data(df_processed: pd.DataFrame, strategy_name: str):
    """전처리된 데이터 분석"""
    print(f"\n{'='*70}")
    print(f"{strategy_name}")
    print(f"{'='*70}")
    
    # 줄바꿈 통계
    newline_counts = df_processed['conversation_processed'].str.count('\n')
    print(f"\n줄바꿈 수 통계:")
    print(f"  - 평균: {newline_counts.mean():.2f}")
    print(f"  - 중앙값: {newline_counts.median():.0f}")
    print(f"  - 최소: {newline_counts.min():.0f}")
    print(f"  - 최대: {newline_counts.max():.0f}")
    print(f"  - 줄바꿈 없음: {(newline_counts == 0).sum():,}개")
    
    # 텍스트 길이 통계
    text_lengths = df_processed['conversation_processed'].str.len()
    print(f"\n텍스트 길이 통계:")
    print(f"  - 평균: {text_lengths.mean():.0f}자")
    print(f"  - 중앙값: {text_lengths.median():.0f}자")
    
    # 샘플 출력
    print(f"\n샘플 예시 (원본 vs 전처리):")
    sample_idx = df_processed[df_processed['conversation'].str.count('\n') > 5].index[0]
    print(f"\n[원본]")
    print(df_processed.loc[sample_idx, 'conversation'][:200] + "...")
    print(f"\n[전처리 후]")
    print(df_processed.loc[sample_idx, 'conversation_processed'][:200] + "...")

# 각 전략별 분석
analyze_processed_data(df_strategy1, "전략 1: 줄바꿈 완전 제거")
analyze_processed_data(df_strategy2, "전략 2: 구분 토큰 대체")
analyze_processed_data(df_strategy3, "전략 3: 일반 대화 줄바꿈 추가")

# ===================================================================
# 5. 데이터 저장
# ===================================================================

print("\n" + "=" * 70)
print("5. 전처리 데이터 저장")
print("=" * 70)

# 각 전략별 저장
output_columns = ['idx', 'class', 'conversation_processed', 'strategy']

df_strategy1_output = df_strategy1[output_columns].rename(
    columns={'conversation_processed': 'conversation'}
)
df_strategy1_output.to_csv('train_strategy1_remove_newlines.csv', index=False, encoding='utf-8-sig')
print("\n✓ 전략 1 저장 완료: train_strategy1_remove_newlines.csv")

df_strategy2_output = df_strategy2[output_columns].rename(
    columns={'conversation_processed': 'conversation'}
)
df_strategy2_output.to_csv('train_strategy2_separator_token.csv', index=False, encoding='utf-8-sig')
print("✓ 전략 2 저장 완료: train_strategy2_separator_token.csv")

df_strategy3_output = df_strategy3[output_columns].rename(
    columns={'conversation_processed': 'conversation'}
)
df_strategy3_output.to_csv('train_strategy3_add_random_newlines.csv', index=False, encoding='utf-8-sig')
print("✓ 전략 3 저장 완료: train_strategy3_add_random_newlines.csv")

# 모든 전략 통합 데이터 저장
df_all_strategies = pd.concat([
    df_strategy1[output_columns],
    df_strategy2[output_columns],
    df_strategy3[output_columns]
], ignore_index=True)

df_all_strategies_output = df_all_strategies.rename(
    columns={'conversation_processed': 'conversation'}
)
df_all_strategies_output.to_csv('train_all_strategies_combined.csv', index=False, encoding='utf-8-sig')
print("✓ 전체 통합 저장 완료: train_all_strategies_combined.csv")

# ===================================================================
# 6. 비교 분석표
# ===================================================================

print("\n" + "=" * 70)
print("6. 전략별 비교 요약")
print("=" * 70)

comparison_data = []

for df_temp, strategy_name in [
    (df_strategy1, '전략 1: 줄바꿈 제거'),
    (df_strategy2, '전략 2: 구분토큰'),
    (df_strategy3, '전략 3: 줄바꿈 추가')
]:
    newline_counts = df_temp['conversation_processed'].str.count('\n')
    text_lengths = df_temp['conversation_processed'].str.len()
    
    comparison_data.append({
        '전략': strategy_name,
        '데이터 수': len(df_temp),
        '평균 줄바꿈': f"{newline_counts.mean():.2f}",
        '줄바꿈 없음': f"{(newline_counts == 0).sum():,}",
        '평균 길이': f"{text_lengths.mean():.0f}자",
        '특징': {
            '전략 1: 줄바꿈 제거': '단일 문맥, 가장 단순',
            '전략 2: 구분토큰': '턴 구조 보존, 토큰 기반',
            '전략 3: 줄바꿈 추가': '데이터 균형, 다양성 증가'
        }[strategy_name]
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n")
print(comparison_df.to_string(index=False))

# ===================================================================
# 7. 완료 메시지
# ===================================================================

print("\n" + "=" * 70)
print("전처리 완료!")
print("=" * 70)
print(f"""
생성된 파일:
1. train_strategy1_remove_newlines.csv      (전략1: {len(df_strategy1):,}개)
2. train_strategy2_separator_token.csv      (전략2: {len(df_strategy2):,}개)  
3. train_strategy3_add_random_newlines.csv  (전략3: {len(df_strategy3):,}개)
4. train_all_strategies_combined.csv        (통합: {len(df_all_strategies):,}개)

권장사항:
- 전략 1: 단순 분류 태스크에 적합
- 전략 2: 턴 기반 대화 모델에 적합 (BERT, RoBERTa 등)
- 전략 3: 데이터 증강 및 균형 확보 시 적합
- 실험을 통해 성능이 가장 좋은 전략 선택 권장
""")


1. 데이터 로드 및 기본 분석

전체 데이터 수: 4,750개
컬럼: ['idx', 'class', 'conversation']

클래스 분포:
class
기타 괴롭힘 대화      1094
갈취 대화           981
직장 내 괴롭힘 대화     979
협박 대화           896
일반              800
Name: count, dtype: int64

줄바꿈 통계:
count    4750.000000
mean        7.781684
std         3.664396
min         0.000000
25%         9.000000
50%         9.000000
75%         9.000000
max        24.000000
Name: newline_count, dtype: float64

- 줄바꿈 없는 데이터: 800개
- 줄바꿈 있는 데이터: 3,950개

2. 전처리 함수 정의

3. 각 전략별 전처리 수행

[전략 1] 줄바꿈 완전 제거 적용 중...
[전략 2] 구분 토큰([SEP]) 대체 적용 중...
[전략 3] 일반 대화 데이터에 줄바꿈 추가 중...

전처리 완료!

4. 전처리 결과 분석

전략 1: 줄바꿈 완전 제거

줄바꿈 수 통계:
  - 평균: 0.00
  - 중앙값: 0
  - 최소: 0
  - 최대: 0
  - 줄바꿈 없음: 4,750개

텍스트 길이 통계:
  - 평균: 204자
  - 중앙값: 185자

샘플 예시 (원본 vs 전처리):

[원본]
지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다. 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다. 정말 잘못했습니다.
 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다. 한...

[전처리 후]
지금 너 스스로를 죽여달라고 애원하는 것